# HistFactory

In [ ]:
%%html
<p align="center">
<iframe src="https://matthewfeickert.github.io/talk-SciPy-2020/index.html#2" width="1200" height="675"></iframe>
</p>

In [ ]:
%%html
<p align="center">
<iframe src="https://matthewfeickert.github.io/talk-SciPy-2020/index.html#6" width="1200" height="675"></iframe>
</p>

In [ ]:
%%html
<p align="center">
<iframe src="https://matthewfeickert.github.io/talk-SciPy-2020/index.html#7" width="1200" height="675"></iframe>
</p>

In [ ]:
%%html
<p align="center">
<iframe src="https://matthewfeickert.github.io/talk-SciPy-2020/index.html#13" width="1200" height="675"></iframe>
</p>

## JSON Spec

Let's investiage this spec a bit more now

```json
{
    "channels": [
        { "name": "singlechannel",
          "samples": [
            { "name": "signal",
              "data": [5.0, 10.0],
              "modifiers": [ { "name": "mu", "type": "normfactor", "data": null} ]
            },
            { "name": "background",
              "data": [50.0, 60.0],
              "modifiers": [ {"name": "uncorr_bkguncrt", "type": "shapesys", "data": [5.0, 12.0]} ]
            }
          ]
        }
    ],
    "observations": [
        { "name": "singlechannel", "data": [50.0, 60.0] }
    ],
    "measurements": [
        { "name": "Measurement", "config": {"poi": "mu", "parameters": []} }
    ],
    "version": "1.0.0"
}
```

> which demonstrates a simple measurement of a single two-bin channel with two samples: a **signal sample** and a **background sample**. The signal sample has an _unconstrained normalisation factor_ $\mu$, while the background sample carries an _uncorrelated shape systematic_ controlled by parameters $\gamma_{1}$ and $\gamma_{2}$. The _background uncertainty_ for the bins is $10\%$ and $20\%$ respectively.

In [ ]:
import pyhf
import json

Let's open up the spec and then load it into a workspace

In [ ]:
with open("data/2-bin_1-channel.json") as serialized:
  spec = json.load(serialized)

workspace = pyhf.Workspace(spec)

and then create a statistical model from it

In [ ]:
model = workspace.model(measurement_name="Measurement")

In [ ]:
# Take a quick look at the model spec
model.spec

Let's clean that up a bit to make it more readable

In [ ]:
def pretty_json(jsonlike, indent=None):
    if indent is None: indent = 4
    print(json.dumps(jsonlike, indent=indent))

In [ ]:
pretty_json(model.spec)

and now actually break that down again.

### Single Channel

> demonstrates a simple measurement of a single two-bin channel

In [ ]:
print(f"Channels in model: {model.config.channels}\n")

single_channel = model.spec["channels"][0]

print(f"Number of bins in channel: {model.config.channel_nbins}\n")

In [ ]:
single_channel

### Samples

> with two samples

In [ ]:
model.config.samples

In [ ]:
pretty_json(single_channel["samples"])

### Modifiers

In [ ]:
model.config.modifiers

> The signal sample has an unconstrained normalisation factor $\mu$

In [ ]:
signal_sample = single_channel["samples"][0]
pretty_json(signal_sample["modifiers"])

> the background sample carries an uncorrelated shape systematic controlled by parameters $\gamma_{1}$ and $\gamma_{2}$

In [ ]:
# Each bin has its own shape systematic
background_sample = single_channel["samples"][1]
pretty_json(background_sample["modifiers"])

> The background uncertainty for the bins is $10\%$ and $20\%$ respectively.

In [ ]:
import numpy as np

bkg_uncert = background_sample["modifiers"][0]["data"]
np.array(bkg_uncert)/np.array(background_sample["data"])

In [ ]:
import requests

remote_url = "https://raw.githubusercontent.com/scikit-hep/pyhf/0f99cc488156e0826a27f55abc946d537a8922af/docs/examples/json/2-bin_1-channel.json"
response = json.loads(requests.get(remote_url).text)

pretty_json(response)

## Observations

The data assocaited with a workspace

In [ ]:
workspace.data(model)

contains both observations

In [ ]:
workspace.data(model, with_aux=False)

as well as axuiliary information

In [ ]:
model.config.auxdata

## Measurements

The parameter(s) of interest (POI) you're trying to measure

In [ ]:
pretty_json(workspace.get_measurement())

## Inspecting workspaces

We can also use the `pyhf` command line tool to help us verify and inspect the workspace

In [ ]:
! pyhf inspect data/2-bin_1-channel.json